In [13]:
import re
import os
import csv
import pandas as pd
import subprocess
from pathlib import Path

class TexProcesser():
    def __init__(self, latex_dir_path, output_path):
        self.latex_folder = Path(latex_dir_path)
        self.output_path = output_path
        
    def pipeline(self,):
        self.processed_tex_dir = self.output_path + '/' + 'processed_tex'
        self._preprocess(input_dir=self.latex_folder, output_dir=self.processed_tex_dir)
        
        output_csv_path = self.output_path + '/latex_info.csv'
        self._extract_field_to_csv(output_csv_path=output_csv_path)
        
        # TODO:把文本批量转换为md文件
        self._postprocess()
        
    def _preprocess(self, input_dir, output_dir):
        '''
        删除一些额外的表格和公式等等
        '''

        for file_path in sorted(input_dir.glob('**/*.tex')):
        # latex = './NLP_LATEX_CORPUS/V04/V04N01-07.tex'
            file_name = os.path.basename(file_path)
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    latex_content = file.read()
                print(f'processing {file_name} ...')
                # 执行其他操作
            except UnicodeDecodeError:
                print(f"UnicodeDecodeError: Failed to decode {file_path}. Skipping this file.")
                continue  # 跳过当前文件的处理
            
            # 删除\begin{document}之前的所有内容
            latex_content = re.sub(r'[\s\S]*?(?=\\begin\{document\})', '', latex_content, count=1, flags=re.DOTALL)
                    
            # 删除bibliographystyle之后的所有文本
            # latex_content = re.sub(r'\\bibliographystyle\{.*?\}[\s\S]*', '', latex_content, flags=re.DOTALL)
            
            # 删除所有的图片
            latex_content = re.sub(r'\\begin{figure}.*?\\end{figure}', '', latex_content, flags=re.DOTALL)
            
            # 删除所有的表格
            latex_content = re.sub(r'\\begin{table}.*?\\end{table}', '', latex_content, flags=re.DOTALL)
            latex_content = re.sub(r'\\begin{tabular}.*?\\end{tabular}', '', latex_content, flags=re.DOTALL)
            
            # 替换引用\ref{...}文本为[...]
            latex_content = re.sub(r'\\ref\{([^{}]*)\}', r'[\1]', latex_content, flags=re.DOTALL)
            
            # 删除格式调整符号
            # latex_content = re.sub(r'\\hspace\*\{[^{}]*\}', '', latex_content, flags=re.DOTALL)
            # latex_content = re.sub(r'\\vspace\*\{[^{}]*\}', '', latex_content, flags=re.DOTALL)        
        
            # 删除所有的标签\label{...}
            latex_content = re.sub(r'\\label\{.*?\}', '', latex_content, flags=re.DOTALL)
            
            # 替换掉文本中有\underline{...}的内容            
            # latex_content = re.sub(r'\\underline\{([^{}]*)\}', r'\1', latex_content, flags=re.DOTALL)            
                                    
            # 替换掉所有文献引用为@xcite
            latex_content = re.sub(r'\\cite\{.*?\}', '@xcite', latex_content, flags=re.DOTALL)
            
            # 将所有的公式都替换为@xmath0, @xmath1, @xmath2, ...
            formula_count = 0
            def replace_formula(match):
                nonlocal formula_count
                new_formula = f'@xmath{formula_count}'
                formula_count += 1
                return new_formula
            
            latex_content = re.sub(r'\$.*?\$', replace_formula, latex_content, flags=re.DOTALL)

            # 删除所有脚注
            # NOTE: 必须在替换公式之后在进行删除，否则会引发冲突
            # latex_content = re.sub(r'\\footnote\{.*?\}', '', latex_content, flags=re.DOTALL)
            # latex_content = re.sub(r'(?s)\\footnote\{.*?\}', '', latex_content)
            # latex_content = re.sub  (r'\\footnotemark', '', latex_content, flags=re.DOTALL)
            # latex_content = re.sub(r'\\footnotetext\{.*?\}', '', latex_content, flags=re.DOTALL)       
            
            # # 使用正则表达式找到最后一个 \section{} 的位置并删除其后的内容
            # match_sections = [m.end() for m in re.finditer(r'\\section\{.*?\}', latex_content)]  # 查找所有 \section{} 的位置
            # if match_sections:
            #     last_section_index = match_sections[-1]  
            #     content_after_section = latex_content[last_section_index:]  # 获取最后一个 \section{} 之后的内容

            #     match_next_unknown = re.search(r'\n\\[a-zA-Z]+\{.*?\}', content_after_section)  # 寻找下一个未知标记
            #     if match_next_unknown:
            #         next_unknown_index = last_section_index + match_next_unknown.start()  # 获取下一个未知标记的位置
            #         latex_content = latex_content[:next_unknown_index]  # 保留标记之前的内容
            
            # # 在文章的末尾添加\end{document}
            # latex_content += '\n\\end{document}'
            
            # 删除\acknowledgment
            # latex_content = re.sub(r'\\acknowledgment.*?(?=\\end\{document\})', '', latex_content, flags=re.DOTALL)
            
            # 将处理后的内容保存到新文件中
            output_file = output_dir + '/' + file_name
            with open(output_file, 'w', encoding='utf-8') as file:
                file.write(latex_content)    
                 
    def _extract_field_to_csv(self, output_csv_path=None):
        '''
        extract title, etitle, jabstract, eabstract ... from latex files
        '''
        
        # 定义csv文件的表头
        output_csv_path = Path(output_csv_path)
        csv_header = ['file_name', 'title', 'etitle', 'jabstract', 'eabstract', 'section_names',
                      'sec_intro', 'sec_method', 'sec_result', 'sec_conclusion',
                      'abs_intro', 'abs_method', 'abs_result', 'abs_conclusion']
        
        with open(output_csv_path, 'w', newline='', encoding='utf-8') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=csv_header)
            writer.writeheader()
            
            for file_path in sorted(self.latex_folder.glob('**/*.tex')):
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        latex_content = file.read()
                    print(f'processing {file_path} ...')
                except UnicodeDecodeError:
                    print(f"UnicodeDecodeError: Failed to decode {file_path}. Skipping this file.")
                    continue
                    
                matches = self._extract_info(latex_content=latex_content)
                matches['file_name'] = file_path.name
                
                extra_columns = [
                    'sec_intro', 'sec_method', 'sec_result', 'sec_conclusion',
                    'abs_intro', 'abs_method', 'abs_result', 'abs_conclusion'
                ]
                for column in extra_columns:
                    matches[column] = ''
                
                writer.writerow(matches)
        
    def _extract_info(self, latex_content=None):
        
        patterns = {
            # TODO: title 和 jtitle 有时候会同时出现，需要处理
            'title': re.compile(r'\\title\{(.*?)\}', re.DOTALL),
            'etitle': re.compile(r'\\etitle\{(.*?)\}', re.DOTALL),
            'jabstract': re.compile(r'\\jabstract\{(.*?)\}', re.DOTALL),
            'eabstract': re.compile(r'\\eabstract\{(.*?)\}', re.DOTALL),     
            'section_names': re.compile(r'\\section\{(.*?)\}', re.DOTALL),                 
        }
        
        matches = {key: '' if key != 'section_names' else [] for key in patterns.keys()}
        
        for key, pattern in patterns.items():
            matches[key] = pattern.findall(latex_content)
            
            if key == 'section_names':
                matches[key] = [value.strip() for value in matches[key]]  # 移除首尾空白并存储在列表中
                matches[key] = list(filter(None, matches[key]))  # 移除空字符串
            
            if key != 'section_names' and matches[key]:
                matches[key] = re.sub(r'\s*\\\\\n\s*', ' ', matches[key][0]).strip()
            elif key != 'section_names':
                matches[key] = ''

        return matches
    
    def _postprocess(self):
        """
        convert all processed tex files to markdown files
        """
        markdown_output_dir = Path(self.output_path) / 'markdown'
        markdown_output_dir.mkdir(parents=True, exist_ok=True)
        
        for file_path in sorted(Path(self.processed_tex_dir).glob('*.tex')):
            md_output_path = markdown_output_dir / file_path.name.replace('.tex', '.txt')

            try:
                subprocess.run(["pandoc", str(file_path), "-o", str(md_output_path)])
                print(f"Converted {file_path} to Markdown: {md_output_path}")
            except Exception as e:
                print(f"Conversion failed for {file_path}: {e}")

In [ ]:
tex_processer = TexProcesser(latex_dir_path='./data/NLP_LATEX_CORPUS/', output_path='./data')
tex_processer.pipeline()

In [ ]:
import pandas as pd

df = pd.read_csv('./data/latex_info.csv')
df.head()

In [8]:
df.to_csv('NLP_JP_CORPUS.csv', index=False, encoding='utf-8-sig')

In [113]:
import pandas as pd

df = pd.read_csv('./NLP_JP_CORPUS_processed.csv')

In [114]:
df.fillna('', inplace=True)
df

,file_name,language,title,etitle,jabstract,eabstract,section_names,sec_intro,sec_method,sec_result,sec_conclusion,abs_bg,abs_method,abs_result,abs_conclusion
0,V01N01-01.tex,jp,表層表現中の情報に基づく文章構造の自動抽出,Automatic Detection of Discourse Structure by ...,テキストや談話を理解するためには，まずその文章構造を理解する必要があ\nる．文章構造に関する...,"To understand a text or dialogue, one must tra...","['はじめに', '文章構造のモデルと結束関係', '文章構造の自動抽出', '実験と考察'...",テキストや談話を理解するためには，文章構造の理解，すなわち各文が\n他のどの文とどのような関...,従来，文章構造のモデルとしてはその基本単位の結束関係\n(2項関係)を再帰的に組み合わせる\...,"実験には科学雑誌サイエンスのテキスト，\n「科学技術のためのコンピューター」(Vol.17,...",本論文では，手がかり表現，語連鎖，文間の類似性，という\n表層表現中の3つ情報に基づいて文章...,テキストや談話を理解するためには，まずその文章構造を理解する必要があ\nる．文章構造に関する...,本論文では，知識に基づく文理解という処理を行なわずに，表層表現中の種々の情報を用いることによ...,実験の結果これらの情報を組み合わせて利用することにより\n科学技術文の文章構造のかなりの部分...,
1,V01N01-02.tex,en,,A Comparative Study of Automatic Extraction of...,,While corpus-based studies are now becoming a ...,"['Introduction', 'Importance of Collocational ...","Recent rapid advances in computer technology, ...","In the past, several approaches have been prop...","In our experiments, the ADD (ATR Dialogue Data...",With the growing availability of large textual...,While corpus-based studies are now becoming a ...,"In this paper, we are primarily concerned with...",Comparative experiments are made between the t...,
2,V01N01-03.tex,jp,並列構造の検出に基づく長い日本語文の構文解析,A Syntactic Analysis Method of Long Japanese S...,従来の構文解析法は十分な精度の解析結果を得ることができず，とくに長い文の解析が困難であった．...,Conventional parsing methods can not analyze l...,"['はじめに', '並列構造の検出と文の簡単化', '係り受け解析', '文解析の結果とその...",# はじめに\n\n従来の構文解析法は基本的に句構造文法あるいは格文法をその拠り所としてきた...,# 並列構造の検出と文の簡単化\n\n## 並列構造の検出の概要\n\n1文中の並列する部分...,# 文解析の結果とその評価\n\n本手法による文解析の実験をテストサンプルの150文に対して...,# おわりに\n\n従来方式の構文解析では長い文の中に多く存在する並列構造を正しく認識する\...,従来の構文解析法は十分な精度の解析結果を得ることができず，とくに長い文の解析が困難であった．...,本論文では，そのようにして検出した並列構造の情報を利用して\n構文解析を行なう手法を示す．\...,各部分の係り受け解析としては，基本的に，\n係り受け関係の非交差条件を満たした上で各文節が係...,我々は，このような考え方に基づき，長い文の中に多く存在する並列構造が\n文節列同士の類似性を...
3,V01N01-04.tex,en,,A System for Finding Translation Patterns by C...,,When the example-based approach is used for ma...,"['Introduction', 'System Overview', 'Example-B...","The example-based approach (EBA), an emerging ...",The system flow of is shown in . An input Jap...,,Strong recent interest in corpus-based process...,When the example-based approach is used for ma...,This\npaper describes a system for finding par...,,
4,V02N01-01.tex,jp,日英機械翻訳における利用者登録語の意味属性の自動推定,Automatic Determination of Semantic Attributes...,"機械翻訳システムを使用して現実の文書を翻訳する場合, 通常, \n翻訳対象文書に合った利用者...",User dictionaries are important for practical ...,"['はじめに', 'システム辞書と利用者辞書', '意味属性推定の方法', '意味属性推定精...","# はじめに\n\n機械翻訳システムを使用する時, 利用者はシステム辞書に登録されていない\...","# 意味属性推定の方法\n\n利用者登録語の日本語表記と英語訳語が与えられたとき, 機械翻訳...",# 意味属性推定精度の評価\n\n## 実験の条件\n\n表\[tab:3\]に示すような新...,## 考察\n\n### 訳文品質向上効果について\n\n最適意味属性を決定する繰り返し実験...,"機械翻訳システムを使用して現実の文書を翻訳する場合, 通常, \n翻訳対象文書に合った利用者...","そこで本論文では, 利用者が登録したい日本語名詞 (複合名詞を含む) と\n英語訳語を与える...","本方式を, 新聞記事102文とソフトウエア設計書105文\nの翻訳に必要な利用者辞書作成に適...","以上の結果, 利用者辞書\n作成への単語の登録において, 最も熟練度の要求される単語意味属性..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,V26N03-03.tex,jp,事前学習された文の分散表現を用いた機械翻訳の自動評価,Metric for Automatic Machine Translation Evalu...,本稿では，参照文を用いた文単位での機械翻訳自動評価手法について述べる．現在のデファクトスタン...,This study describes a segment-level metric fo...,"['はじめに', '関連研究', '事前学習された文の分散表現を用いた機械翻訳の自動評価',...",# はじめに\n\n本稿では，参照文を用いた文単位での機械翻訳自動評価手法について述べる．\...,# 事前学習された文の分散表現を用いた機械翻訳の自動評価\n\n従来手法に多く見られる文字や...,# 評価実験\n\n本節では，WMT Metrics Shared\nTaskにおける人手の...,# 分析\n\n## 訓練データの文対数と性能の関係\n\n本節では，WMT-2017のlv...,本稿では，参照文を用いた文単位での機械翻訳自動評価手法について述べる．現在のデファクトスタン...,そこで本研究では，文全体の大域的な情報を考慮するために，事前学習された文の分散表現を用いる機...,WMT-2017 Metrics Shared Taskにおける翻訳品質のラベル付きデータセ...,
528,V26N03-04.tex,jp,単語埋め込みに基づくサプライザル,Surprisal through Word Embeddings,ヒトの文処理のモデル化として Hale によりサプライザルが提案されている．サプライザルは文...,The concept of surprisal was proposed by Hale ...,"['はじめに', '前提', '分析手法', '結果と考察', 'おわりに', '分析結果（...",# はじめに\n\n本研究では眼球運動に基づき文の読み時間を推定し，ヒトの文処理機構の解明を...,# 分析手法\n\n分析においては，いくつかの要因に基づく線形式に基づいて，読み

In [115]:
jp_df = df[df['language']=='jp']
jp_df

,file_name,language,title,etitle,jabstract,eabstract,section_names,sec_intro,sec_method,sec_result,sec_conclusion,abs_bg,abs_method,abs_result,abs_conclusion
0,V01N01-01.tex,jp,表層表現中の情報に基づく文章構造の自動抽出,Automatic Detection of Discourse Structure by ...,テキストや談話を理解するためには，まずその文章構造を理解する必要があ\nる．文章構造に関する...,"To understand a text or dialogue, one must tra...","['はじめに', '文章構造のモデルと結束関係', '文章構造の自動抽出', '実験と考察'...",テキストや談話を理解するためには，文章構造の理解，すなわち各文が\n他のどの文とどのような関...,従来，文章構造のモデルとしてはその基本単位の結束関係\n(2項関係)を再帰的に組み合わせる\...,"実験には科学雑誌サイエンスのテキスト，\n「科学技術のためのコンピューター」(Vol.17,...",本論文では，手がかり表現，語連鎖，文間の類似性，という\n表層表現中の3つ情報に基づいて文章...,テキストや談話を理解するためには，まずその文章構造を理解する必要があ\nる．文章構造に関する...,本論文では，知識に基づく文理解という処理を行なわずに，表層表現中の種々の情報を用いることによ...,実験の結果これらの情報を組み合わせて利用することにより\n科学技術文の文章構造のかなりの部分...,
2,V01N01-03.tex,jp,並列構造の検出に基づく長い日本語文の構文解析,A Syntactic Analysis Method of Long Japanese S...,従来の構文解析法は十分な精度の解析結果を得ることができず，とくに長い文の解析が困難であった．...,Conventional parsing methods can not analyze l...,"['はじめに', '並列構造の検出と文の簡単化', '係り受け解析', '文解析の結果とその...",# はじめに\n\n従来の構文解析法は基本的に句構造文法あるいは格文法をその拠り所としてきた...,# 並列構造の検出と文の簡単化\n\n## 並列構造の検出の概要\n\n1文中の並列する部分...,# 文解析の結果とその評価\n\n本手法による文解析の実験をテストサンプルの150文に対して...,# おわりに\n\n従来方式の構文解析では長い文の中に多く存在する並列構造を正しく認識する\...,従来の構文解析法は十分な精度の解析結果を得ることができず，とくに長い文の解析が困難であった．...,本論文では，そのようにして検出した並列構造の情報を利用して\n構文解析を行なう手法を示す．\...,各部分の係り受け解析としては，基本的に，\n係り受け関係の非交差条件を満たした上で各文節が係...,我々は，このような考え方に基づき，長い文の中に多く存在する並列構造が\n文節列同士の類似性を...
4,V02N01-01.tex,jp,日英機械翻訳における利用者登録語の意味属性の自動推定,Automatic Determination of Semantic Attributes...,"機械翻訳システムを使用して現実の文書を翻訳する場合, 通常, \n翻訳対象文書に合った利用者...",User dictionaries are important for practical ...,"['はじめに', 'システム辞書と利用者辞書', '意味属性推定の方法', '意味属性推定精...","# はじめに\n\n機械翻訳システムを使用する時, 利用者はシステム辞書に登録されていない\...","# 意味属性推定の方法\n\n利用者登録語の日本語表記と英語訳語が与えられたとき, 機械翻訳...",# 意味属性推定精度の評価\n\n## 実験の条件\n\n表\[tab:3\]に示すような新...,## 考察\n\n### 訳文品質向上効果について\n\n最適意味属性を決定する繰り返し実験...,"機械翻訳システムを使用して現実の文書を翻訳する場合, 通常, \n翻訳対象文書に合った利用者...","そこで本論文では, 利用者が登録したい日本語名詞 (複合名詞を含む) と\n英語訳語を与える...","本方式を, 新聞記事102文とソフトウエア設計書105文\nの翻訳に必要な利用者辞書作成に適...","以上の結果, 利用者辞書\n作成への単語の登録において, 最も熟練度の要求される単語意味属性..."
5,V02N01-02.tex,jp,主観的動機に関する意味および語用論的制約を利用した日本語複文の理解システム 〜「ので」「のに...,A zero anaphora resolution systemfor Japanese ...,我々は，接続助詞「ので」による順接の複文と接続助詞「のに」による逆接の複\n文を対象とする理...,Our aim is to construct a system which is able...,"['はじめに', '日本語複文に関する制約および素性構造による表現', '制約変換による日本...",# はじめに\n\n我々が目標とするのは，日本語の複文の理解システムである．このようなシステ...,# 日本語複文に関する制約および素性構造による表現\n\n本論文で述べるシステムにより意味解...,# 制約変換による日本語複文の意味解析システム\n\n以上のように，意味および語用論的役割の...,# おわりに\n\n本論文では順接の「ので」による複文の解析を例にとり，日本語の複文の意味解...,我々は，接続助詞「ので」による順接の複文と接続助詞「のに」による逆接の複\n文を対象とする理...,この際には，ゼロ代名詞の照応の解析が重要な問題となるが，文献にあるように，本論文で扱う形式の...,そこで，日本語の複文に対する形態素解析や構文解析の結果を素性構造で記述し，こ\nの結果に対し...,
6,V02N01-03.tex,jp,文章内構造を複合的に利用した論説文要約システムGREEN,GREEN: An Experimental System Generating Summa...,"日本語文章要約システムについて報告する. 一般に, 質の良い文章要約を\n行うためには, あ...",This paper describes an experimental system fo...,"['はじめに', 'システム構成', '要約文選択', '文要約解析', '段落分け解析',...",# はじめに\n\n本論文では日本語の論説文を対象にした要約文章作成実験システム[^1] (...,"# システム構成\n\n要約システム は̏ Sun SPARC Station I 上で, ...","# 文要約解析\n\n文中の修飾句を削減することにより, 一文内での要約を行う. 文の中心内...","# 議論\n\nここでは, 機械処理した大量の要約結果の考察から得られた知見や明らかになっ\...","日本語文章要約システムについて報告する. 一般に, 質の良い文章要約を\n行うためには, あ...","本研究ではこの観点から, 日本語での様々な表層的特徴をできるだけ多く利用\nして, 日本語文...","本稿では実際に計算機上で試作した論説文\n要約システムに関して, これで用いられている論説文...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,V26N03-03.tex,jp,事前学習された文の分散表現を用いた機械翻訳の自動評価,Metric for Automatic Machine Translation Evalu...,本稿では，参照文を用いた文単位での機械翻訳自動評価手法について述べる．現在のデファクトスタン...,This study describes a segment-level metric fo...,"['はじめに', '関連研究', '事前学習された文の分散表現を用いた機械翻訳の自動評価',...",# はじめに\n\n本稿では，参照文を用いた文単位での機械翻訳自動評価手法について述べる．\...,# 事前学習された文の分散表現を用いた機械翻訳の自動評価\n\n従来手法に多く見られる文字や...,# 評価実験\n\n本節では，WMT Metrics Shared\nTaskにおける人手の...,# 分析\n\n## 訓練データの文対数と性能の関係\n\n本節では，WMT-2017のlv...,本稿では，参照文を用いた文単位での機械翻訳自動評価手法について述べる．現在のデファクトスタン...,そこで本研究では，文全体の大域的な情報を考慮するために，事前学習された文の分散表現を用いる機...,WMT-2017 Metrics Shared Taskにおける翻訳品質のラベル付きデータセ...,
528,V26N03-04.tex,jp,単語埋め込みに基づくサプライザ

In [116]:
df.rename(columns={'abs_bg': 'abs_intro'}, inplace=True)

In [117]:
jp_df.rename(columns={'abs_bg': 'abs_intro'}, inplace=True)

/tmp/ipykernel_2013502/981833588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jp_df.rename(columns={'abs_bg': 'abs_intro'}, inplace=True)


In [118]:
jp_df_copy = jp_df.copy()

In [ ]:
jp_df_copy

In [120]:
# 对文本列进行post processing
import re
import pandas as pd

def process_text(text):
    # 删除换行符 \n
    text = text.replace('\n', '')
    
    # 删除 {.underline}
    text = text.replace("{.underline}", "")

    # 删除 \underline
    text = text.replace("\\underline", "")
    
    # 删除 \begin{enumerate} 和 \end{enumerate} 以及 \item
    text = re.sub(r'\\begin\{enumerate\}|\\end\{enumerate\}|\\item', '', text)
    
    # 更换网址为 @url
    text = re.sub(r'(http|https)://[^\s]+', '@url', text)    
    
    # 删除::: 开头的文本（带有空格）
    text = re.sub(r':::\s(sent|center|table\*|flashleft|flashright|exe|algorithm\(ic\)|indent|xlist|lingexample|'
                  r'figure\*|ex|exB|description|itembox|DEPEX|breakbox|df|enumerate|savenotes|dependency|'
                  r'algorithm(ic)?|簡体中文)\s?', '', text)
    # 删除:::本身
    text = text.replace(':::', '')
    
    # 匹配$$之间的数学公式
    pattern = r'\$\$(.*?)\$\$'
    
    # 使用正则表达式替换数学公式为 @xlmath
    text = re.sub(pattern, r'@xlmath', text)
    
    # 删除\
    text = text.replace('\\', '')
    
    return text

In [121]:
jp_df_copy['sec_method']

0      従来，文章構造のモデルとしてはその基本単位の結束関係\n(2項関係)を再帰的に組み合わせる\...
2      # 並列構造の検出と文の簡単化\n\n## 並列構造の検出の概要\n\n1文中の並列する部分...
4      # 意味属性推定の方法\n\n利用者登録語の日本語表記と英語訳語が与えられたとき, 機械翻訳...
5      # 日本語複文に関する制約および素性構造による表現\n\n本論文で述べるシステムにより意味解...
6      # システム構成\n\n要約システム は̏ Sun SPARC Station I 上で, ...
                             ...                        
527    # 事前学習された文の分散表現を用いた機械翻訳の自動評価\n\n従来手法に多く見られる文字や...
528    # 分析手法\n\n分析においては，いくつかの要因に基づく線形式に基づいて，読み時間をベイジ...
529    # MWEを考慮した依存構造コーパスの構築\n\n本章では，英語の複合機能語および形容詞MW...
530    # 提案手法\n\n本研究では，所与の文脈を手がかりとして，各単語に品詞 \@xciteやト...
531    # 単語報酬モデル\n\n本章では，提案する単語報酬モデルの各要素について説明する．\n単語...
Name: sec_method, Length: 414, dtype: object

In [123]:
non_string_indices = jp_df_copy[~jp_df_copy['sec_method'].apply(lambda x: isinstance(x, str))].index
jp_df_copy.drop(non_string_indices, inplace=True)
print(jp_df_copy[['file_name', 'sec_method']])
non_string_indices = jp_df_copy[~jp_df_copy['sec_result'].apply(lambda x: isinstance(x, str))].index
jp_df_copy.drop(non_string_indices, inplace=True)
print(jp_df_copy[['file_name', 'sec_method']])
non_string_indices = jp_df_copy[~jp_df_copy['abs_method'].apply(lambda x: isinstance(x, str))].index
# jp_df_copy.drop(non_string_indices, inplace=True)
print(jp_df_copy[['file_name', 'sec_method']])

         file_name                                         sec_method
0    V01N01-01.tex  従来，文章構造のモデルとしてはその基本単位の結束関係\n(2項関係)を再帰的に組み合わせる\...
2    V01N01-03.tex  # 並列構造の検出と文の簡単化\n\n## 並列構造の検出の概要\n\n1文中の並列する部分...
4    V02N01-01.tex  # 意味属性推定の方法\n\n利用者登録語の日本語表記と英語訳語が与えられたとき, 機械翻訳...
5    V02N01-02.tex  # 日本語複文に関する制約および素性構造による表現\n\n本論文で述べるシステムにより意味解...
6    V02N01-03.tex  # システム構成\n\n要約システム は̏ Sun SPARC Station I 上で, ...
..             ...                                                ...
527  V26N03-03.tex  # 事前学習された文の分散表現を用いた機械翻訳の自動評価\n\n従来手法に多く見られる文字や...
528  V26N03-04.tex  # 分析手法\n\n分析においては，いくつかの要因に基づく線形式に基づいて，読み時間をベイジ...
529  V26N04-01.tex  # MWEを考慮した依存構造コーパスの構築\n\n本章では，英語の複合機能語および形容詞MW...
530  V26N04-02.tex  # 提案手法\n\n本研究では，所与の文脈を手がかりとして，各単語に品詞 \@xciteやト...
531  V26N04-03.tex  # 単語報酬モデル\n\n本章では，提案する単語報酬モデルの各要素について説明する．\n単語...

[414 rows x 2 columns]
         file_name                                         sec_method
0    V01N01-01.tex  従来，文章構造のモデルとしてはその基本単位の結束関係\n(2項関係)を再帰的に組み合わせる\

In [124]:
jp_df_copy

,file_name,language,title,etitle,jabstract,eabstract,section_names,sec_intro,sec_method,sec_result,sec_conclusion,abs_intro,abs_method,abs_result,abs_conclusion
0,V01N01-01.tex,jp,表層表現中の情報に基づく文章構造の自動抽出,Automatic Detection of Discourse Structure by ...,テキストや談話を理解するためには，まずその文章構造を理解する必要があ\nる．文章構造に関する...,"To understand a text or dialogue, one must tra...","['はじめに', '文章構造のモデルと結束関係', '文章構造の自動抽出', '実験と考察'...",テキストや談話を理解するためには，文章構造の理解，すなわち各文が\n他のどの文とどのような関...,従来，文章構造のモデルとしてはその基本単位の結束関係\n(2項関係)を再帰的に組み合わせる\...,"実験には科学雑誌サイエンスのテキスト，\n「科学技術のためのコンピューター」(Vol.17,...",本論文では，手がかり表現，語連鎖，文間の類似性，という\n表層表現中の3つ情報に基づいて文章...,テキストや談話を理解するためには，まずその文章構造を理解する必要があ\nる．文章構造に関する...,本論文では，知識に基づく文理解という処理を行なわずに，表層表現中の種々の情報を用いることによ...,実験の結果これらの情報を組み合わせて利用することにより\n科学技術文の文章構造のかなりの部分...,
2,V01N01-03.tex,jp,並列構造の検出に基づく長い日本語文の構文解析,A Syntactic Analysis Method of Long Japanese S...,従来の構文解析法は十分な精度の解析結果を得ることができず，とくに長い文の解析が困難であった．...,Conventional parsing methods can not analyze l...,"['はじめに', '並列構造の検出と文の簡単化', '係り受け解析', '文解析の結果とその...",# はじめに\n\n従来の構文解析法は基本的に句構造文法あるいは格文法をその拠り所としてきた...,# 並列構造の検出と文の簡単化\n\n## 並列構造の検出の概要\n\n1文中の並列する部分...,# 文解析の結果とその評価\n\n本手法による文解析の実験をテストサンプルの150文に対して...,# おわりに\n\n従来方式の構文解析では長い文の中に多く存在する並列構造を正しく認識する\...,従来の構文解析法は十分な精度の解析結果を得ることができず，とくに長い文の解析が困難であった．...,本論文では，そのようにして検出した並列構造の情報を利用して\n構文解析を行なう手法を示す．\...,各部分の係り受け解析としては，基本的に，\n係り受け関係の非交差条件を満たした上で各文節が係...,我々は，このような考え方に基づき，長い文の中に多く存在する並列構造が\n文節列同士の類似性を...
4,V02N01-01.tex,jp,日英機械翻訳における利用者登録語の意味属性の自動推定,Automatic Determination of Semantic Attributes...,"機械翻訳システムを使用して現実の文書を翻訳する場合, 通常, \n翻訳対象文書に合った利用者...",User dictionaries are important for practical ...,"['はじめに', 'システム辞書と利用者辞書', '意味属性推定の方法', '意味属性推定精...","# はじめに\n\n機械翻訳システムを使用する時, 利用者はシステム辞書に登録されていない\...","# 意味属性推定の方法\n\n利用者登録語の日本語表記と英語訳語が与えられたとき, 機械翻訳...",# 意味属性推定精度の評価\n\n## 実験の条件\n\n表\[tab:3\]に示すような新...,## 考察\n\n### 訳文品質向上効果について\n\n最適意味属性を決定する繰り返し実験...,"機械翻訳システムを使用して現実の文書を翻訳する場合, 通常, \n翻訳対象文書に合った利用者...","そこで本論文では, 利用者が登録したい日本語名詞 (複合名詞を含む) と\n英語訳語を与える...","本方式を, 新聞記事102文とソフトウエア設計書105文\nの翻訳に必要な利用者辞書作成に適...","以上の結果, 利用者辞書\n作成への単語の登録において, 最も熟練度の要求される単語意味属性..."
5,V02N01-02.tex,jp,主観的動機に関する意味および語用論的制約を利用した日本語複文の理解システム 〜「ので」「のに...,A zero anaphora resolution systemfor Japanese ...,我々は，接続助詞「ので」による順接の複文と接続助詞「のに」による逆接の複\n文を対象とする理...,Our aim is to construct a system which is able...,"['はじめに', '日本語複文に関する制約および素性構造による表現', '制約変換による日本...",# はじめに\n\n我々が目標とするのは，日本語の複文の理解システムである．このようなシステ...,# 日本語複文に関する制約および素性構造による表現\n\n本論文で述べるシステムにより意味解...,# 制約変換による日本語複文の意味解析システム\n\n以上のように，意味および語用論的役割の...,# おわりに\n\n本論文では順接の「ので」による複文の解析を例にとり，日本語の複文の意味解...,我々は，接続助詞「ので」による順接の複文と接続助詞「のに」による逆接の複\n文を対象とする理...,この際には，ゼロ代名詞の照応の解析が重要な問題となるが，文献にあるように，本論文で扱う形式の...,そこで，日本語の複文に対する形態素解析や構文解析の結果を素性構造で記述し，こ\nの結果に対し...,
6,V02N01-03.tex,jp,文章内構造を複合的に利用した論説文要約システムGREEN,GREEN: An Experimental System Generating Summa...,"日本語文章要約システムについて報告する. 一般に, 質の良い文章要約を\n行うためには, あ...",This paper describes an experimental system fo...,"['はじめに', 'システム構成', '要約文選択', '文要約解析', '段落分け解析',...",# はじめに\n\n本論文では日本語の論説文を対象にした要約文章作成実験システム[^1] (...,"# システム構成\n\n要約システム は̏ Sun SPARC Station I 上で, ...","# 文要約解析\n\n文中の修飾句を削減することにより, 一文内での要約を行う. 文の中心内...","# 議論\n\nここでは, 機械処理した大量の要約結果の考察から得られた知見や明らかになっ\...","日本語文章要約システムについて報告する. 一般に, 質の良い文章要約を\n行うためには, あ...","本研究ではこの観点から, 日本語での様々な表層的特徴をできるだけ多く利用\nして, 日本語文...","本稿では実際に計算機上で試作した論説文\n要約システムに関して, これで用いられている論説文...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,V26N03-03.tex,jp,事前学習された文の分散表現を用いた機械翻訳の自動評価,Metric for Automatic Machine Translation Evalu...,本稿では，参照文を用いた文単位での機械翻訳自動評価手法について述べる．現在のデファクトスタン...,This study describes a segment-level metric fo...,"['はじめに', '関連研究', '事前学習された文の分散表現を用いた機械翻訳の自動評価',...",# はじめに\n\n本稿では，参照文を用いた文単位での機械翻訳自動評価手法について述べる．\...,# 事前学習された文の分散表現を用いた機械翻訳の自動評価\n\n従来手法に多く見られる文字や...,# 評価実験\n\n本節では，WMT Metrics Shared\nTaskにおける人手の...,# 分析\n\n## 訓練データの文対数と性能の関係\n\n本節では，WMT-2017のlv...,本稿では，参照文を用いた文単位での機械翻訳自動評価手法について述べる．現在のデファクトスタン...,そこで本研究では，文全体の大域的な情報を考慮するために，事前学習された文の分散表現を用いる機...,WMT-2017 Metrics Shared Taskにおける翻訳品質のラベル付きデータセ...,
528,V26N03-04.tex,jp,単語埋め込みに基づくサプ

In [125]:
jp_df_copy['sec_intro'] = jp_df_copy['sec_intro'].apply(process_text)
jp_df_copy['abs_intro'] = jp_df_copy['abs_intro'].apply(process_text)

jp_df_copy['sec_method'] = jp_df_copy['sec_method'].apply(process_text)
jp_df_copy['abs_method'] = jp_df_copy['abs_method'].apply(process_text)

jp_df_copy['sec_result'] = jp_df_copy['sec_result'].apply(process_text)
jp_df_copy['abs_result'] = jp_df_copy['abs_result'].apply(process_text)

jp_df_copy['sec_conclusion'] = jp_df_copy['sec_conclusion'].apply(process_text)
jp_df_copy['abs_conclusion'] = jp_df_copy['abs_conclusion'].apply(process_text)

In [128]:
jp_df['abs_conclusion'][39]

'その際，最良解釈を求めるために必要な処理だけを行ない，それ以外の処理の\n実行は必要が生じるまで保留することによって無駄な処理を避ける．\n保留した処理を必要に応じて再開することによって，最良解釈以外の解釈も選\nび出せる．'

In [129]:
jp_df_copy['abs_conclusion'][39]

'その際，最良解釈を求めるために必要な処理だけを行ない，それ以外の処理の実行は必要が生じるまで保留することによって無駄な処理を避ける．保留した処理を必要に応じて再開することによって，最良解釈以外の解釈も選び出せる．'

In [ ]:
jp_df_copy.to_csv('NLP_JP_CORPUS_jp_only_processed.csv', index=False, encoding='utf-8-sig')